In [1]:
# Import required libraries
import pandas as pd
import numpy as np

In [2]:
#Creating a function to automatically identify possible predictor ICD10 codes. Use this function in other notebooks.
def get_predictor_codes(edges, exclusions, input_code, limit = 20):
    filtered_edges = edges.loc[edges['Edge'].str.contains(input_code)].sort_values(by = 'Weight', ascending = False).reset_index(drop=True)

    # If there are exclusions remove edges where the other base code is in the exclusion list 
    if len(exclusions) != 0:
        filtered_edges = filtered_edges.loc[~filtered_edges['Edge'].str.contains('|'.join(exclusions))].reset_index(drop=True)



    # Return the other base code of the pair
    other_codes = [''] * limit
    other_desc = [''] * limit
    weights = [''] * limit

    for i in range(limit):
        current_source = filtered_edges['Source'].iloc[i]
        current_source_desc = filtered_edges['Source Description'].iloc[i]
        current_target = filtered_edges['Target'].iloc[i]
        current_target_desc = filtered_edges['Target Description'].iloc[i]

        if current_source == input_code:
            other_codes[i] = current_target
            other_desc[i] = current_target_desc
        else:
            other_codes[i] = current_source
            other_desc[i] = current_source_desc
        weights[i] = filtered_edges['Weight'].iloc[i]
    return (other_codes, other_desc, weights)


In [3]:
# Importing the formatted dataset and the exclusions dataset
all_data = pd.read_csv('Data/formatted_data_all_codes.csv')
all_data = all_data.drop('Unnamed: 0', axis=1)

In [4]:
# Remove member life IDs that have conflicting Biological Genders recorded.
# Get the Member Life IDs with two or more rows
gender_df = all_data[['Member Life ID', 'Biological Gender']]

gender_df = gender_df.loc[gender_df['Member Life ID'].duplicated(keep=False)].drop_duplicates()

ids_with_two_genders = gender_df[gender_df['Member Life ID'].duplicated(keep=False)]['Member Life ID'].drop_duplicates().to_list()

all_data = all_data[~all_data['Member Life ID'].isin(ids_with_two_genders)]
all_data = all_data.reset_index(drop=True)

In [5]:
all_data

,Member Life ID,100,123,130,160,230,240,250,300,367,...,Z94,Z95,Z96,Z97,Z98,Z99,Biological Gender,Line Service From Date,Header Service From Date,Birth Date
0,109514,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,F,2017-03-02,2017-03-02,1984-12-18
1,11824943,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,F,2017-04-06,2017-04-06,NaN
2,2573644,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,F,2017-01-03,2017-01-03,NaN
3,2602510,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,F,2017-02-09,2017-02-09,1959-12-04
4,32741106,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,F,2017-01-04,2017-01-04,1990-11-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100032,455440,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,F,2017-04-21,2017-04-21,NaN
100033,226218,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,M,2017-02-16,2017-02-16,1943-06-03
100034,24552166,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,M,2018-01-30,2018-01-30,1989-12-30
100035,18835833,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,M,2017-10-05,2017-10-05,1955-08-26


In [6]:
# Create an all_data_with_ages dataframe
# Rows with Ages Only (Assuming that the Header Service From Date is the one to use)
all_data_with_ages = all_data.loc[pd.notna(all_data['Birth Date'])]
all_data_with_ages = all_data_with_ages.drop('Line Service From Date', axis = 1)
all_data_with_ages[['Header Service From Date', 'Birth Date']] = all_data_with_ages[['Header Service From Date', 'Birth Date']].apply(pd.to_datetime)
all_data_with_ages['Age'] = np.floor((all_data_with_ages['Header Service From Date'] - all_data_with_ages['Birth Date']).dt.days/365.25)
all_data_with_ages = all_data_with_ages.drop(['Header Service From Date', 'Birth Date'], axis=1)
all_data_with_ages = all_data_with_ages.drop_duplicates()
all_data_with_ages = all_data_with_ages.reset_index(drop=True)

In [7]:
# Removing Member Life IDs that have more than one age
# Get the Member Life IDs with two or more rows
age_df = all_data_with_ages[['Member Life ID', 'Age']]

age_df = age_df.loc[age_df['Member Life ID'].duplicated(keep=False)].drop_duplicates()

ids_with_two_ages = age_df[age_df['Member Life ID'].duplicated(keep=False)]['Member Life ID'].drop_duplicates().to_list()

all_data_with_ages = all_data_with_ages[~all_data_with_ages['Member Life ID'].isin(ids_with_two_ages)]
all_data_with_ages = all_data_with_ages.reset_index(drop=True)

In [8]:
all_data_with_ages

,Member Life ID,100,123,130,160,230,240,250,300,367,...,Z92,Z93,Z94,Z95,Z96,Z97,Z98,Z99,Biological Gender,Age
0,109514,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,F,32.0
1,2602510,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,F,57.0
2,32741106,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,F,26.0
3,32735876,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,F,37.0
4,3360996,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,F,59.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53807,13282371,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,F,26.0
53808,3916556,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,F,28.0
53809,226218,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,M,73.0
53810,24552166,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,M,28.0


In [9]:
# Write the all_data_with_ages dataframe to a csv
all_data_with_ages.to_csv('all_data_with_ages.csv')